In [ ]:
import pathlib
import os
import warnings
warnings.filterwarnings('ignore')

# Change to the project root directory
project_root = pathlib.Path("/Users/victormp/Desktop/ml/ml-project")
os.chdir(project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.arima.model import ARIMA as Arima

# Import Data

In [ ]:
data_path = pathlib.Path("data/DCOILWTICO.csv")
print(f"Loading from: {data_path.absolute()}")
print(f"File exists: {data_path.exists()}")

data = pd.read_csv(data_path)
data.rename(columns = {"observation_date" : "date", "DCOILWTICO" : "price"}, inplace  = True)
data["date"] = pd.to_datetime(data["date"])
data = data.set_index("date")
data["return"] = np.log(data["price"]) - np.log(data["price"].shift(1))
returns = data["return"].replace([np.inf, -np.inf], np.nan).dropna().astype("float32")

# Plot Data

In [ ]:
plt.figure(figsize=(14, 5))
sns.lineplot(x = data.index, y = data["price"], alpha=0.3)
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()

# Split Data

In [ ]:
split_point = int(len(returns) * 0.8)  # 80% train, 20% test
y_train = returns.iloc[:split_point]
y_test = returns.iloc[split_point:]

# Convert y_train["return"] and y_test["return"] to darts TimeSeries objects
train_series = TimeSeries.from_values(y_train)
test_series = TimeSeries.from_values(y_test)

# AR Model Selection

In [ ]:
returns = y_train
print("="*60)
print("AR vs ARMA MODEL COMPARISON")
print("="*60)
print(f"Sample size: {len(returns)}\n")

# Test different combinations
results = []
print(f"{'Model':<15} {'AIC':<12} {'BIC':<12} {'Log-Lik':<12}")
print("-"*60)

#Pure AR models
for p in range(1, 6):
    try:
        model = Arima(returns, order=(p, 0, 0)).fit()
        results.append({
            'model': f'AR({p})',
            'p': p, 'q': 0,
            'aic': model.aic,
            'bic': model.bic,
            'llf': model.llf
        })
        print(f"AR({p}){'':<10} {model.aic:<12.2f} {model.bic:<12.2f} {model.llf:<12.2f}")
    except Exception as e:
        print(f"AR({p}) FAILED: {e}")
        continue

print()

#ARMA models
for p in range(1, 4):
    for q in range(1, 4):
        try:
            model = Arima(returns, order=(p, 0, q)).fit()
            results.append({
                'model': f'ARMA({p},{q})',
                'p': p, 'q': q,
                'aic': model.aic,
                'bic': model.bic,
                'llf': model.llf
            })
            print(f"ARMA({p},{q}){'':<8} {model.aic:<12.2f} {model.bic:<12.2f} {model.llf:<12.2f}")
        except Exception as e:
            print(f"ARMA({p},{q}) FAILED: {e}")
            continue

if len(results) == 0:
    print("\nERROR: No models could be fitted!")
    raise ValueError("Cannot proceed - no models were fitted successfully")

#find best models
results_df = pd.DataFrame(results)
best_aic_idx = results_df['aic'].idxmin()
best_bic_idx = results_df['bic'].idxmin()
best_aic_model = results_df.loc[best_aic_idx, 'model']
best_bic_model = results_df.loc[best_bic_idx, 'model']

print("\n" + "="*60)
print("BEST MODELS:")
print(f"  By AIC: {best_aic_model:<12} (AIC = {results_df.loc[best_aic_idx, 'aic']:.2f})")
print(f"  By BIC: {best_bic_model:<12} (BIC = {results_df.loc[best_bic_idx, 'bic']:.2f})")
print("="*60)

#Check if ARMA is better
best_ar_bic = results_df[results_df['q'] == 0]['bic'].min()
best_arma_bic = results_df[results_df['q'] > 0]['bic'].min()

print("\nCONCLUSION:")
if best_arma_bic < best_ar_bic:
    improvement = best_ar_bic - best_arma_bic
    print(f"  ARMA fits better than pure AR")
    print(f"  BIC improvement: {improvement:.2f}")
    print(f"  Best model: {best_bic_model}")
else:
    print(f"  Pure AR is sufficient")
    best_ar_idx = results_df[results_df['q']==0]['bic'].idxmin()
    print(f"  Best model: {results_df.loc[best_ar_idx, 'model']}")
print("="*60)

# Fit and diagnose best model
print(f"\nFitting best model: {best_bic_model}")

# Extract p and q properly
best_p = int(results_df.loc[best_bic_idx, 'p'])
best_q = int(results_df.loc[best_bic_idx, 'q'])

# Now fit the model
final_model = Arima(returns, order=(best_p, 0, best_q)).fit()
print("\nParameter Estimates:")
print(final_model.summary().tables[1])

# Check residuals
residuals = final_model.resid
lb_test = acorr_ljungbox(residuals, lags=[5, 10], return_df=True)
print("\nResidual Diagnostics (Ljung-Box):")
print(lb_test[['lb_stat', 'lb_pvalue']])

if (lb_test['lb_pvalue'] > 0.05).all():
    print("\n Residuals are white noise (model is adequate)")
else:
    print("\n Some autocorrelation remains in residuals")


#Extended Ljung-Box test with 20 lags for autocorrelation
print("\n" + "="*60)
print("Extended Ljung-Box test")
print("="*60)

lb_extended = acorr_ljungbox(residuals, lags=range(1, 21), return_df=True)
print("\nLjung-Box Test Results (lags 1-20):")
print(lb_extended[['lb_stat', 'lb_pvalue']])

# Summary
significant_lags = lb_extended[lb_extended['lb_pvalue'] < 0.05]
if len(significant_lags) == 0:
    print("\n Residuals pass Ljung-Box test at all lags")
else:
    print(f"\n Autocorrelation detected at {len(significant_lags)} lag(s):")
    print(f"  Lags: {list(significant_lags.index)}")

# Store the best model info for later use
best_model_order = (best_p, 0, best_q)
print(f"\nBest model order to use: {best_model_order}")